### The Cell below imports the functions and packages needed for ThellierCoolPy to run

In [ ]:
%%capture 

from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)

### Use this next cell to import the function file to the cloud. This is ThellierCoolFunc_Colab.py 

In [ ]:
getLocalFiles()

### Use this next cell to import the zip file which must contain the FORC data (.frc) file, the uncorrected Thellier (.tdt) files and if measured Ms-T curve data will be used, the Ms-T curve data (.dat) file. If no Ms-T curve data (.dat) file is found, you will be asked to manually import a Curie temperature at a later step.

In [ ]:
import ThellierCoolFunc_Colab as tf
getLocalFiles()

### The cell below will open the zip file and processes the FORC data. The raw data files will be saved to a folder called 'raw_data'. The cell takes ~ 30 seconds to run.

In [ ]:
if ('X' in locals()) == False:
    X = {}

V = {} 
tf.open_zip_file()
X, V, curve = tf.find_FORC_MsT(X, V)
X = tf.proccess_all(X)
X = tf.prod_FORCs(X)

### The next two cells calculate the full width at half maximum (FWHM). This is used in calculating the FORC distribution at SF = 0.

In [ ]:
X = tf.finding_fwhm(X)

In [ ]:
X = tf.plot_fwhm_1(X) 

### This section is for checking for anomalous FWHM. If a point lies far from the line of best fit on the graph above remove the respective SF here. This line should have a slight positive gradient. 

In [ ]:
X = tf.check_fwhm(X) 

### Enter the smallest smoothing factor which was accepted during the FWHM section above 

In [ ]:
SF = (input("Input the lowest reliable SF from the FWHM graph above:" )) #make this automatic to be smallest posisble?
X['SF'] = int(SF)

### This cell plots the FORC diagram with the smallest accepted smoothing factor


In [ ]:
X = tf.divide_mu0(X) #do i need this divide - havent checked in a while
X = tf.sym_norm_forcs(X)
tf.norm_rho_all(X)
tf.plot_sample_FORC(X['Hc'], X['Hu'], X['rho_n'], X['SF'], X['name'])

### Input the lab cooling time in hours, the natural cooling time in hours. If there is no Ms-T data (.dat) file in your original zip folder, you will also be asked to input a Curie temperature for the sample in &deg;C. Also, using the displayed FORC diagram above please select the absolute maximum Hc and Hi where the FORC distribution is not zero to increase the model efficiency by only picking points with non-zero values.

In [ ]:
X, V = tf.user_input(X, V, curve)

### This cell will correct the Thellier files (.tdt) and produce a zipped folder (processed_data.zip) which contains the corrected Thellier files (original_file_name_U.tdt) along with the the FORC diagram (forc_file_name.pdf). This will take ~ 3 minutes.

In [ ]:
tf.TRM_acq(X, V, curve)
V = tf.plots_correction_factor(V,X)
tf.fix_files(V)
tf.plot_sample_FORC2(X['Hc'], X['Hu'], X['rho_n'], X['SF'], X['name'],X['reset_limit_hc'],X['reset_limit_hi'])